In [166]:
import pandas as pd
import numpy as np

Data preparation

In [167]:
df_raw = pd.read_csv('./Data_Processed_All_Contractors_Characteristics - Data.csv')

In [168]:
columns_list = list(df_raw.columns)

columns_list.index('applied 2016')

55

In [169]:
#extract 2016 registed lines
df_2016 = df_raw.loc[df_raw['registed 2016'] == 1]
df_2016.head()

,Dealer ID,Converted State-AL,Converted State-AR,Converted State-CA,Converted State-FL,Converted State-GA,Converted State-IL,Converted State-IN,Converted State-KY,Converted State-LA,...,2016_other,2017_summer,2017_winter,2017_other,2018_summer,2018_winter,2018_other,2019_summer,2019_winter,2019_other
0,C15306,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
1,C15309,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,C15310,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,C15311,0,0,0,0,0,0,0,0,0,...,5,0,0,1,0,0,0,0,0,0
4,C15312,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [170]:
#delete unrelated columns
df_2016_xy = df_2016.iloc[:,:56].drop(['Dealer ID','registed 2016','JS 17.Column3','JS 18.Column3','JS 19.Column3',\
                                                         'NON USER 17.Column3','NON USER 18.Column3','Velocity 17.Column3','centurty 18.Column3',\
                                                         'Gibson 18.Column3','employeebucket-NA','employeebucket-4~8','employeebucket-<4',\
                                                         'employeebucket->8'],axis=1)
print(df_2016_xy.isnull().any())

Converted State-AL                          False
Converted State-AR                          False
Converted State-CA                          False
Converted State-FL                          False
Converted State-GA                          False
Converted State-IL                          False
Converted State-IN                          False
Converted State-KY                          False
Converted State-LA                          False
Converted State-MI                          False
Converted State-MO                          False
Converted State-NC                          False
Converted State-NJ                          False
Converted State-OH                          False
Converted State-Others                      False
Converted State-PA                          False
Converted State-SC                          False
Converted State-TN                          False
Converted State-TX                          False
No. of Employees                             True


In [171]:
df_2016_xy.columns

Index(['Converted State-AL', 'Converted State-AR', 'Converted State-CA',
       'Converted State-FL', 'Converted State-GA', 'Converted State-IL',
       'Converted State-IN', 'Converted State-KY', 'Converted State-LA',
       'Converted State-MI', 'Converted State-MO', 'Converted State-NC',
       'Converted State-NJ', 'Converted State-OH', 'Converted State-Others',
       'Converted State-PA', 'Converted State-SC', 'Converted State-TN',
       'Converted State-TX', 'No. of Employees', 'Sales between 0-99,999',
       'Sales between 100,000-499,999', 'Other Sales', 'Sales N/A',
       'Currently offers Consumer Financing?_No',
       'Currently offers Consumer Financing?_Yes',
       'Currently offers Consumer Financing?_N/A', 'Over 10', 'Below 10',
       'No year info', 'Hitting Potential 16.Column3', 'JS 16.Column3',
       'NON USER 16.Column3', 'brand_1', 'brand_2', 'brand_3', 'brand_4',
       'brand_5', 'brand_6', 'brand_7', 'brand_8', 'applied 2016'],
      dtype='object')

In [173]:
#drop NaN rows

narow=[]
for i in range(df_2016_xy.shape[0]):
    if df_2016_xy["No. of Employees"].isnull()[i]== True:
        narow.append(i)

df_2016_xy_dropna=df_2016_xy.drop(narow,axis=0)
df_2016_xy_dropna["No. of Employees"].isnull().any()

False

In [174]:
x = df_2016_xy_dropna.iloc[:,:41]
y = df_2016_xy_dropna.iloc[:,41] .astype(int)
print(y.dtype)
x.head()

int64


,Converted State-AL,Converted State-AR,Converted State-CA,Converted State-FL,Converted State-GA,Converted State-IL,Converted State-IN,Converted State-KY,Converted State-LA,Converted State-MI,...,JS 16.Column3,NON USER 16.Column3,brand_1,brand_2,brand_3,brand_4,brand_5,brand_6,brand_7,brand_8
0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,1,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,1,0


Build logistic regression model (with cv)

In [175]:
import numpy as np
# from sklearn.linear_model.logistic import LogisticRegression
from sklearn.linear_model.logistic import LogisticRegressionCV #model building
from sklearn.preprocessing import StandardScaler #standardize xs
from sklearn.model_selection import train_test_split #train_test_split: spilt train and test dataset; GridSearchCV: cross validation
from sklearn.metrics import confusion_matrix #evaluation: confusion matrix

In [176]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=0)
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [177]:
logreg = LogisticRegressionCV(solver='liblinear')
logreg.fit(X_train, Y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='liblinear', tol=0.0001,
           verbose=0)

In [178]:
acc = logreg.score(X_test_std,Y_test)
acc

0.5753424657534246

In [179]:
pred = logreg.predict(X_test_std)
pred

array([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,

In [180]:
matrix = confusion_matrix(Y_test, pred, labels=None, sample_weight=None)
confusion_matrix = pd.DataFrame(matrix,columns=['pred_true','pred_false'],index=['real_true','real_false'])
confusion_matrix

,pred_true,pred_false
real_true,153,60
real_false,126,99


In [181]:
precision =  153/(153+126) #pred_true中real_true
recall = 153/(153+60) #real_true中pred_true
accuracy=(153+99)/(153+99+60+126) #pred中pred==real
F1 = 2*(153)/(2*153+60+126)
print("precision:"+str(precision)+"\nrecall:"+str(recall)+"\naccuracy:"+str(accuracy)+"\nF1:"+str(F1))

precision:0.5483870967741935
recall:0.7183098591549296
accuracy:0.5753424657534246
F1:0.6219512195121951
